In [10]:
import numpy as np
from numpy import dot
import scipy.linalg as la
import sklearn
from sklearn.cross_decomposition import CCA as sklearn_CCCA
from sklearn import decomposition
from sklearn.utils.extmath import randomized_svd
import time

In [45]:
class CCAModel(object):

        def __init__(self, dim):
        
                self.mean_x, self.mean_y, self.A, self.B, self.sum_crr = None, None, None, None, None
                self.dim = dim
                
        def __call__(self, H1, H2 = None, training = True):
        
                # H1 and H2 are featurs X num_points matrices containing samples columnwise.
                # dim is the desired dimensionality of CCA space.

                if training and (H2 is None):
                        
                                raise Exception("Expected two views in training.")
    
                if training:

                        N = H1.shape[1] 
                        
                        # Remove mean
                        
                        m1 = np.mean(H1, axis=1)
                        m2 = np.mean(H2, axis=1)
                        self.mean_x, self.mean_y = m1, m2
                        
                        H1 = H1 - m1[:, None]
                        H2 = H2 - m2[:, None]
    
                        S11 = (H1.dot(H1.T))/(N-1) # cov_xx
                        S22 = (H2.dot(H2.T))/(N-1) # cov_yy
                        S12 = H2.dot(H1.T)/(N-1) # cov_yx
                        
                        # calculate K11 = inverse(sqrt(S11)), K22 = inverse(sqrt(S22))
                        
                        D1,V1 = la.eig(S11)
                        D2,V2 = la.eig(S22)

                        K11 = V1.dot(np.diag(1/np.sqrt(D1))).dot(V1.T)
                        K22 = V2.dot(np.diag(1/np.sqrt(D2))).dot(V2.T))

                        
                        # Calculate correlation matrix
                        
                        T = K22.dot(S12).dot(K11)
                        
                        # Perform SVD on correlation matrix
                        
                        U,D,V = np.linalg.svd(T)
                        self.corr = np.mean(D)
                        U, V = U[:self.dim, :], V[:self.dim, :]
                        D = np.diag(D) # correlation coefficiens of the canonical components
                        A = dot(K11,V.T) # projection matrix for H1
                        B = dot(K22,U.T) # projection matrix for H2
                        
                        self.A, self.B = A,B
                        
                        
                        # Project & return
                        H1_proj, H2_proj = H1.T.dot(self.A), H2.T.dot(self.B)
                        return H1_proj, H2_proj, self.corr
                
                else:
                        # in test time, use saved mean and projection matrix.
                        
                        H1 -= self.mean_x[:, None]
                        return H1.T.dot(self.A)

In [46]:
def create_dataset(size = 2000, dim = 50):
    
    X = np.random.rand(size, dim) - 0.5
    Y = 1.5 * X + 0.1 * (np.random.rand(*X.shape) - 0.5) # Y is linearly correlated with X + some noise
    
    return X,Y

In [47]:
cca_dim = 3
X,Y = create_dataset(size = 2000, dim = 50)

***Perform gold CCA.***

In [48]:
model = sklearn_CCCA(n_components=cca_dim, tol = 1e-8, max_iter= 50000)
model.fit(X,Y)
x,y = model.transform(X,Y)
print("Projected X first elements:\n {} \n\n Projected Y first elemenets:\n {}".format(x[:4],y[:4]))

Projected X first elements:
 [[ 0.57367207  1.45814387 -0.09837337]
 [-0.75973449  1.46924012 -1.949711  ]
 [ 1.59093368 -0.98325315 -0.237967  ]
 [-0.05101062 -0.64054106  0.48598452]] 

 Projected Y first elemenets:
 [[ 0.55917569  1.43136325 -0.05600422]
 [-0.85103198  1.42718596 -1.98022189]
 [ 1.62507462 -0.96377369 -0.13568865]
 [-0.01936337 -0.64042943  0.47896145]]


***Perform CCA with numpy ***

In [49]:
model = CCAModel(dim = cca_dim)
x,y, corr = model(H1=X.T,H2=Y.T, training = True)
x,y = np.real(x), np.real(y)
print("Projected X first elements:\n {} \n\n Projected Y first elemenets:\n {}".format(x[:4],y[:4]))
print("Correlation: {}".format(corr))

Projected X first elements:
 [[-1.33666355 -0.22314026 -1.08169637]
 [-0.23042993 -0.12436398  0.44316865]
 [-0.14744537  1.49643563 -0.90694871]
 [ 0.40185982 -0.01081162  0.72672613]] 

 Projected Y first elemenets:
 [[-0.71872863  0.87757    -1.67268318]
 [ 0.19520301 -2.26149533  0.13432258]
 [-1.09484534  0.83719782  0.67735054]
 [-0.12347293  0.08399104 -0.85785667]]
Correlation: 0.9977808294901362
